# User Interface for Thermostat Smart Connector

This notebook is a knowledge base that acts as a user interface for the thermostat smart connector. It uses ipywidgets to construct an interface for setting the desired temperature and humidity, as well as viewing the actual live temperature and humidity.

The interface includes a slider for setting the desired temperature and a vertical progress bar for viewing the actual live temperature. It also includes a slider for setting the desired humidity and a vertical progress bar for viewing the actual live humidity. When the submit button is clicked, two post knowledge interactions: "post-desired-temp" and "post-desired-humidity" are sent to the knowledge engine.

This notebook also has two react knowledge interactions that react to the live temperature and humidity measurements sent by the earlier described smart connector notebook. These interactions handle the measurements by updating the respective UI progress bars to view the changing values.

In summary, this notebook provides an interactive user interface for controlling and monitoring the thermostat through the smart connector and knowledge engine. It allows users to set the desired temperature and humidity, view the actual live temperature and humidity.

In [1]:
# Thermostat UI2

In [2]:
import logging
import time
import uuid

import helpers as sp
import ipywidgets as widgets
import requests
from IPython.display import display
from ipywidgets import Button, HBox, IntProgress, IntSlider, Label, Layout, Output, VBox
from utils import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ThermostatUI")

In [3]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
# prefixes of ontologies used in the graph patterns
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
}
# temp measurement graph pattern
TEMP_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""
# graph pattern describing the change of the ?desired_temp setting of the thermostat
TEMP_SETTING_GRAPH_PATTERN = """?setting rdf:type saref:SetLevelCommand .
                            ?setting saref:hasValue ?desired_temp .
                            ?setting saref:isMeasuredIn saref:TemperatureUnit .
                            ?setting saref:hasTimestamp ?timestamp .
                            ?setting saref:isCommandOf ?room_id .
                            ?setting saref:relatesToProperty saref:Temperature .
                            ?setting saref:commandIssuedBy ?device_id .
                            """
# humidity measurement graph pattern
HUMIDITY_MEAS_GRAPH_PATTERN = """EDIT THIS"""
# graph pattern describing the change of the ?desired_humidity setting of the thermostat
HUMIDITY_SETTING_GRAPH_PATTERN = """EDIT THIS"""

In [4]:
# this function get triggered to handle the temperature react ki. The progres bar and label get updated with values from the binding
def handle_temp_react_measurements(bindings):
    for binding in bindings:
        temp_progress.value = binding["temp"]
        label_actual_temp.value = f"{ binding['temp'] } ℃"
    return []

In [5]:
# this function get triggered to handle the humidity react ki. The progres bar and label get updated with values from the binding
def handle_humidity_react_measurements(bindings):
    for binding in bindings:
        pass  # EDIT THIS
    return []

In [6]:
import threading


def start_ui_kb(kb_id, kb_name, kb_description, ke_endpoint):
    # delete in case allready exists
    delete_knowledge_base(kb_id, ke_endpoint)

    # register kb
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    # registering temp react ki
    temp_react_measurements_ki = register_react_knowledge_interaction(
        TEMP_MEAS_GRAPH_PATTERN,
        None,
        "react-temp-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    # registering temp post ki
    global temp_post_ki_id  # this is set to global bc we use it in button_click post
    temp_post_ki_id = register_post_knowledge_interaction(
        TEMP_SETTING_GRAPH_PATTERN,
        None,
        "post-desired-temp",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    # registering humidity react ki
    humidity_react_measurements_ki = "EDIT THIS"

    # registering humidity post ki
    global humidity_post_ki_id  # this is set to global bc we use it in button_click post
    humidity_post_ki_id = "EDIT THIS"

    # staring the handle loop for react ki in a separate thread
    thread = threading.Thread(
        target=start_handle_loop,
        args=(
            {
                temp_react_measurements_ki: handle_temp_react_measurements,
                # ADD SOMETHING HERE
            },
            kb_id,
            ke_endpoint,
        ),
    )
    thread.start()


start_ui_kb(
    "http://example.org/ui",
    "UI",
    "UI for measurement",
    "http://knowledge_engine:8280/rest/",
)

2023-06-27 14:27:32 INFO {"messageType":"error","message":"Deletion of knowledge base failed, because it could not be found."}
2023-06-27 14:27:34 INFO registered UI
2023-06-27 14:27:35 INFO received issued knowledge interaction id: http://example.org/ui/interaction/react-temp-measurements
2023-06-27 14:27:35 INFO received issued knowledge interaction id: http://example.org/ui/interaction/post-desired-temp


In [7]:
# NOTHING TO CHNAGE HERE!
# constructing the ipywidget
# temp
label_actual_temp = widgets.Label(value="", orientation="vertical")
temp_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=50,
    step=1,
    description="",
    bar_style="info",
    orientation="vertical",
)
desired_temp_slider = IntSlider(value=0, min=0, max=50, orientation="vertical")
label_desired_temp = widgets.Label(value="", orientation="vertical")

temp_box = HBox(
    [
        label_actual_temp,
        temp_progress,
        desired_temp_slider,
    ]
)
# humidity
label_actual_humidity = widgets.Label(value="", orientation="vertical")
humidity_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description="",
    bar_style="info",
    orientation="vertical",
)
desired_humidity_slider = IntSlider(value=0, min=0, max=100, orientation="vertical")
label_desired_humidity = widgets.Label(value="", orientation="vertical")

humidity_box = HBox(
    [
        label_actual_humidity,
        humidity_progress,
        desired_humidity_slider,
    ]
)
out = Output()
submit_button = Button(description="Submit", button_style="danger")

In [8]:
def button_click(
    desired_temp,
    temp_post_ki_id,
    desired_humidity,
    humidity_post_ki_id,
    kb_id,
    ke_endpoint,
):
    with out:
        out.clear_output()
        logger.info(f"Posting desired temp: {desired_temp}")
        # performing temp post ki with the follwing bindings. Notice that each ?variable in the post graph pattern is present in the binding
        post(
            [
                {
                    "setting": f"<{THERMOSTAT_API_URL}/settings/{str(uuid.uuid4())}>",
                    "desired_temp": f"{desired_temp}",
                    "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
                    "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                    "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
                }
            ],
            temp_post_ki_id,
            kb_id,
            ke_endpoint,
        )

        # UNCOMMENT AND EDIT <[{"EDIT": "THIS"}]>
        # logger.info(f"Posting desired humidity: {desired_humidity}")
        # performing humidity post ki with the follwing bindings. Notice that each ?variable in the post graph pattern is present in the binding

        # post(
        #     [{"EDIT": "THIS"}],
        #     humidity_post_ki_id,
        #     kb_id,
        #     ke_endpoint,
        # )


# we set the button to call the button_click function whenever clicked.
submit_button.on_click(
    lambda x: button_click(
        desired_temp,
        temp_post_ki_id,
        desired_humidity,
        humidity_post_ki_id,
        "http://example.org/ui",
        "http://knowledge_engine:8280/rest/",
    )
)

In [9]:
# DO NOT EDIT!
desired_temp = 0
desired_humidity = 0


def update_desired_temp(change):
    global desired_temp
    # setting desired_temp to whatever the slider is set to
    desired_temp = change.new


def update_desired_humidity(change):
    global desired_humidity
    # setting desired_temp to whatever the slider is set to
    desired_humidity = change.new


# this makes the desired_temp update to what the sliders value is
desired_temp_slider.observe(update_desired_temp, names="value")
desired_humidity_slider.observe(update_desired_humidity, names="value")

In [10]:
# DO NOT EDIT!
display(
    VBox(
        [
            HBox(
                [
                    temp_box,
                    humidity_box,
                ]
            ),
            submit_button,
            out,
        ]
    )
)